In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
# import ast
# import time
# import glob

# Step 10

Add the distance from zebra crossing as another feature for the data. Assume that pedestrians crros the road mostly on the zebra crossing so it gives a "cue feature".

We take the files from folder 07_data_with_ttd

Assume the the z-crossing center coordinates are equal in all recording and has a value of:<br> z_crossing_xy_meters = [61.5, -28.1]  

### Load dfs

In [3]:
# Specify the directory to load files from
directory = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\07_data_with_ttc'
# Get a list of files in the directory
file_name_list = os.listdir(directory)
file_name_list

['18_data_with_ttc.csv',
 '19_data_with_ttc.csv',
 '20_data_with_ttc.csv',
 '21_data_with_ttc.csv',
 '22_data_with_ttc.csv',
 '23_data_with_ttc.csv',
 '24_data_with_ttc.csv',
 '25_data_with_ttc.csv',
 '26_data_with_ttc.csv',
 '27_data_with_ttc.csv',
 '28_data_with_ttc.csv',
 '29_data_with_ttc.csv']

In [4]:
recordingId_list = file_name_list.copy()
for i in range(len(recordingId_list)):
    recordingId_list[i]=recordingId_list[i].replace('_data_with_ttc.csv', '')
recordingId_list

['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [5]:
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"
# Load dataframes
def load_dfs(file_name_list, root_dir, folder_name):
    os.chdir(root_dir)
    # change to data dir
    os.chdir(folder_name)
    dfs = []
    for i in range(len(file_name_list)):
        df=pd.read_csv(file_name_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    return dfs

dfs = load_dfs(file_name_list, root_dir, directory)

In [6]:
dfs[0][50:56]

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,yVelocity,class,pairs,num_of_pairs,hx,hy
50,18,1,15,15,31.00424,-23.09288,247.97471,0.5,0.5,-0.44268,-1.29066,pedestrian,"[[15, 'car', 4.8948071034290415]]",1,-0.375016,-0.927018
51,18,1,16,16,30.91226,-23.35728,246.74548,0.5,0.5,-0.44018,-1.30680,pedestrian,"[[16, 'car', 4.737702048363741]]",1,-0.394816,-0.918760
52,18,1,17,17,30.82306,-23.62592,245.38691,0.5,0.5,-0.43738,-1.29402,pedestrian,"[[17, 'car', inf]]",1,-0.416489,-0.909141
53,18,1,18,18,30.73382,-23.88268,243.88222,0.5,0.5,-0.44174,-1.25762,pedestrian,"[[18, 'car', inf]]",1,-0.440218,-0.897891
54,18,1,19,19,30.64550,-24.12597,242.23012,0.5,0.5,-0.46116,-1.22255,pedestrian,"[[19, 'car', inf]]",1,-0.465922,-0.884826
55,18,1,20,20,30.55316,-24.36426,240.44714,0.5,0.5,-0.50905,-1.19544,pedestrian,"[[20, 'car', inf]]",1,-0.493226,-0.869901


### Define function to calculate distance and add to df

In [7]:
z_crossing_xy_meters = [61.5, -28.1] 
zebra_x = z_crossing_xy_meters[0] 
zebra_y = z_crossing_xy_meters[1]  

def calculate_distance_from_zebra(df):
    # Calculate the distance for rows where class == 'pedestrian'
    df['distance2zebra'] = np.where(df['class'] == 'pedestrian', 
        np.sqrt(    (df['xCenter'] - zebra_x)**2 + (df['yCenter'] - zebra_y)**2    ),  # Euclidean distance
        np.nan  # Set distance to NaN for non-pedestrian rows
    )
    return df

dfs_with_distance = []
for x in tqdm(range(len(dfs))):
    dfs_with_distance.append(calculate_distance_from_zebra(dfs[x]))

100%|██████████| 12/12 [00:00<00:00, 323.15it/s]


In [8]:
len(dfs_with_distance)

12

In [9]:
dfs_with_distance[4]

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,yVelocity,class,pairs,num_of_pairs,hx,hy,distance2zebra
0,22,0,0,0,43.28069,-24.78654,264.37977,1.92513,4.51749,-1.35812,-3.61562,car,[],0,-0.097934,-0.995193,NaN
1,22,0,1,1,42.99497,-25.50532,258.68761,1.92513,4.51749,-1.59720,-3.46023,car,[],0,-0.196158,-0.980572,NaN
2,22,0,2,2,42.64966,-26.18718,252.51434,1.92513,4.51749,-1.85341,-3.24261,car,[],0,-0.300467,-0.953792,NaN
3,22,0,3,3,42.24810,-26.80684,246.65441,1.92513,4.51749,-2.08825,-2.99432,car,[],0,-0.396276,-0.918131,NaN
4,22,0,4,4,41.80333,-27.37893,241.13780,1.92513,4.51749,-2.28349,-2.74239,car,[],0,-0.482705,-0.875783,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56006,22,561,6154,35,83.53767,-12.87451,187.09915,0.50000,0.50000,-1.54762,-0.20327,pedestrian,[],0,-0.992334,-0.123587,26.785713
56007,22,561,6155,36,83.22760,-12.91742,187.42995,0.50000,0.50000,-1.54977,-0.21771,pedestrian,[],0,-0.991604,-0.129314,26.506590
56008,22,561,6156,37,82.91924,-12.96272,187.76616,0.50000,0.50000,-1.55626,-0.23129,pedestrian,[],0,-0.990828,-0.135130,26.228250
56009,22,561,6157,38,82.60572,-13.01038,188.10271,0.50000,0.50000,-1.56482,-0.24489,pedestrian,[],0,-0.990017,-0.140948,25.945097


### Save to CSV

In [10]:
### save to csv files  ###
os.chdir(root_dir)

def save_dfs(dfs_with_ttc_and_distance2zebra, recordingId_list):
    # create folder to save files 
    folder_name='10_data_with_ttc_and_distance_2_zebra'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # change to working dir
    os.chdir(folder_name)
    
    # initialize a list of the files to return by the function
    file_name_data_list=[]
    
    # iterate dfs_with_ttc and save
    for x in tqdm( range(len(dfs_with_ttc_and_distance2zebra)) , desc="Saving CSV files" ):
        # save as CSV file
        file_name=str(recordingId_list[x]) + '_data_with_ttc_and_distance_2_zebra' + '.csv'
        dfs[x].to_csv(file_name, index=False)
        # append the file name to the list
        file_name_data_list.append(file_name)
        
    print(f"files saved to: {os.getcwd()}")
    # change back to parent directory
    os.chdir('..')

    return file_name_data_list

file_name_data_list = save_dfs(dfs_with_distance, recordingId_list)

Saving CSV files: 100%|██████████| 12/12 [00:04<00:00,  2.55it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\10_data_with_ttc_and_distance_2_zebra


In [11]:
file_name_data_list

['18_data_with_ttc_and_distance_2_zebra.csv',
 '19_data_with_ttc_and_distance_2_zebra.csv',
 '20_data_with_ttc_and_distance_2_zebra.csv',
 '21_data_with_ttc_and_distance_2_zebra.csv',
 '22_data_with_ttc_and_distance_2_zebra.csv',
 '23_data_with_ttc_and_distance_2_zebra.csv',
 '24_data_with_ttc_and_distance_2_zebra.csv',
 '25_data_with_ttc_and_distance_2_zebra.csv',
 '26_data_with_ttc_and_distance_2_zebra.csv',
 '27_data_with_ttc_and_distance_2_zebra.csv',
 '28_data_with_ttc_and_distance_2_zebra.csv',
 '29_data_with_ttc_and_distance_2_zebra.csv']